In [1]:
import os
os.chdir('/Users/schoens/Documents/PhD/IIASA/04_Code/income_decile')

In [2]:
from pathlib import Path

import pandas as pd 
import numpy as np
import xarray as xr

from tqdm.notebook import tqdm

import config.config as cfg 
from config.config import (PROCESSED_GMT_FILE,
                           GRAPHICS_DIR, GRAPHICS_PAPER_MAIN_DIR, GRAPHICS_PAPER_APPENDIX_DIR, 
                           RESULTS_DIR,
                           )
from utils.plotting_functions import lighten_color
from utils.processing_functions import regional_data, abbrevs

import regionmask 
ar6 = regionmask.defined_regions.ar6.land


In [3]:
processed_extremes_frequency_df = pd.DataFrame(columns =  ['emitter_region', 'emitter_group', 'event_defintion', 'variable', 'month', 'impacted_region_id', 'impacted_region_abbrev', 'quantile', 'events'])

var_names = ['tas', 'spei']
var_property = 'frequency'
for var_name in var_names: 
    if var_name == 'tas': 
        extreme_type = 'hot'
    if var_name == 'spei':
        extreme_type = 'dry'
    df = xr.open_dataset(RESULTS_DIR / f'{var_name}_{var_property}_{extreme_type}.nc').stack(coords = ('lat', 'lon')).dropna('coords', how = 'all').isel(extreme_quantile = 1)
    df_agg = regional_data(df).quantile(q = [0.05, 0.5, 0.95], dim = 'model')
    del df 
    # additional events since 1990
    df_tmp1 = df_agg[f'1990_{var_name}'].to_dataframe().reset_index().rename(columns = {'mask': 'impacted_region_id', f'1990_{var_name}': 'events'})
    df_tmp1['emitter_region'] = 'World' 
    df_tmp1['emitter_group'] = 'all'
    df_tmp1['event_defintion'] = '1-in-100'
    df_tmp1['variable'] = var_name
    df_tmp1['impacted_region_abbrev'] = df_tmp1['impacted_region_id'].apply(lambda x: abbrevs(x))
    # additional attributable events since 1990 
    df_tmp2 = df_agg[f'attributed_{var_name}'].to_dataframe().reset_index().rename(columns = {'mask': 'impacted_region_id', f'attributed_{var_name}': 'events', 'region': 'emitter_region', 'group': 'emitter_group'})
    df_tmp2['event_defintion'] = '1-in-100'
    df_tmp2['variable'] = var_name
    df_tmp2['impacted_region_abbrev'] = df_tmp2['impacted_region_id'].apply(lambda x: abbrevs(x))

    processed_extremes_frequency_df = pd.concat([processed_extremes_frequency_df, df_tmp1, df_tmp2], axis = 0, ignore_index = True)

processed_extremes_frequency_df.to_csv(RESULTS_DIR / 'processed_extremes_frequency.csv', index = False)

In [4]:
processed_extremes_intensity_df = pd.DataFrame(columns =  ['emitter_region', 'emitter_group', 'event_defintion', 'variable', 'month', 'impacted_region_id', 'impacted_region_abbrev', 'quantile', 'change'])

var_names = ['tas', 'spei']
var_property = 'intensity'
for var_name in var_names: 
    if var_name == 'tas': 
        extreme_type = 'hot'
    if var_name == 'spei':
        extreme_type = 'dry'
    df = xr.open_dataset(RESULTS_DIR / f'{var_name}_{var_property}_{extreme_type}.nc').stack(coords = ('lat', 'lon')).dropna('coords', how = 'all').isel(extreme_quantile = 1)
    df_agg = regional_data(df).quantile(q = [0.05, 0.5, 0.95], dim = 'model')
    del df 
    # additional events since 1990
    df_tmp1 = df_agg[f'1990_{var_name}'].to_dataframe().reset_index().rename(columns = {'mask': 'impacted_region_id', f'1990_{var_name}': 'change'})
    df_tmp1['emitter_region'] = 'World' 
    df_tmp1['emitter_group'] = 'all'
    df_tmp1['event_defintion'] = '1-in-100'
    df_tmp1['variable'] = var_name
    df_tmp1['impacted_region_abbrev'] = df_tmp1['impacted_region_id'].apply(lambda x: abbrevs(x))
    # additional attributable events since 1990 
    df_tmp2 = df_agg[f'attributed_{var_name}'].to_dataframe().reset_index().rename(columns = {'mask': 'impacted_region_id', f'attributed_{var_name}': 'change', 'region': 'emitter_region', 'group': 'emitter_group'})
    df_tmp2['event_defintion'] = '1-in-100'
    df_tmp2['variable'] = var_name
    df_tmp2['impacted_region_abbrev'] = df_tmp2['impacted_region_id'].apply(lambda x: abbrevs(x))

    processed_extremes_intensity_df = pd.concat([processed_extremes_intensity_df, df_tmp1, df_tmp2], axis = 0, ignore_index = True)

processed_extremes_intensity_df.to_csv(RESULTS_DIR / 'processed_extremes_intensity.csv', index = False)